In [11]:
import torch
from torch import nn
from models import InterpretableMultiHeadAttention

In [12]:
embedding_size = 12
d_model = embedding_size
n_head = 4
batch_size = 12
number_features = 6

att_layer = InterpretableMultiHeadAttention(n_head=n_head, d_model=d_model, dropout=0.5)

In [18]:
x = torch.randn(batch_size, number_features, embedding_size)
print(x.shape)

torch.Size([12, 6, 12])


In [17]:
output, att = att_layer(x, x, x)
print(output.shape)
print(att.shape)

torch.Size([12, 6, 12])
torch.Size([12, 6, 4, 6])


In [24]:
att[0,:,0,:]

tensor([[0.0826, 0.4295, 0.1512, 0.1275, 0.0038, 0.2054],
        [0.0444, 0.0023, 0.0639, 0.0276, 0.4900, 0.3718],
        [0.0616, 0.0107, 0.0398, 0.0459, 0.7944, 0.0477],
        [0.1123, 0.3543, 0.0136, 0.3218, 0.1962, 0.0019],
        [0.1376, 0.0558, 0.1141, 0.1252, 0.4247, 0.1426],
        [0.0203, 0.0018, 0.0700, 0.0125, 0.0361, 0.8594]],
       grad_fn=<SliceBackward0>)